In [7]:
from sklearn.linear_model import SGDClassifier
import numpy as np
from simulation_util import client_update
import warnings

# hide the warning message temporarily
warnings.simplefilter("ignore")

## Client Update Example

In [8]:
# this data will be provided by the server
features = [[1, 4, 3], [0, 2, 2], [1, 4, 0], [0, 5, 3], [1, 2, 1], [0, 2, 9]]
labels = [1, 0, 1, 0, 1, 0]
weights = [29., 0., 0., -9]
epochs = 3
batch_size = 3

new_weights = client_update(weights, epochs, batch_size, features, labels)
print(new_weights)

[28.48292577  0.          0.         -9.        ]


## Server Update

In [9]:
import numpy as np
from simulation_util import server_update

init_weights = [0, 0, 0, 0]
client_fraction = 0.5
num_rounds = 10
epoch = 10
batch_size = 25
display_weight_per_round = True

num_client = 100
samples_per_client = 100
num_features = 3
features = np.random.randint(10, size=(num_client, samples_per_client, num_features))
labels = np.random.randint(2, size=(num_client, samples_per_client))

new_clf = server_update(init_weights, client_fraction, num_rounds, features, labels, epoch, batch_size,display_weight_per_round)

[ -2.66061216   2.93237909   1.80483436 -11.20790433]
[-1.90416862 -5.71537901 -5.20763278 -7.7919691 ]
[-4.91891312 -2.26699931  3.31331808 -9.66818133]
[-3.4096222  -1.34464241  0.95563875 -8.12388829]
[ -3.48178403  -3.33094927   0.79192907 -11.57486889]
[ -3.059062    -8.00281514  -1.63996297 -13.43384358]
[-3.59687846  3.00554652  3.04388509 -8.72932695]
[-3.15760823  3.38742142 -0.79405131 -5.91385613]
[ 1.83941204 -5.08778508 -1.16693419 -6.25308119]
[ -5.99501003   0.73437751  -7.9399362  -14.94290568]


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)